### A360AI
Ensure that the A360AI object is loaded

In [ ]:
a360ai

This is the project associated with the workspace.

In [ ]:
a360ai.project_name

This is the workspace flavor. It can be any one of 'spark', 'scipy', 'pytorch', 'tensorflow', 'huggingface', or 'xgboost'.

In [ ]:
import os
image_flavor = os.environ["A360_IMAGE"].split("-")
flavor = image_flavor[-1]
flavor

### Data Repos
Data Repos are A360AI objects used to read and write datasets. A project can be associated with multiple datarepos but a datarepo is always associated with a single project.

List the datarepos associated with the current project.

In [ ]:
datarepos = a360ai.list_datarepos()

if datarepos.shape[0] == 0:
    raise Exception(f"No data repos associated with the project {a360ai.project_name}. Please add a datarepo.")
datarepos

#### Select a datarepo
Please set the *datarepo_name* variable to a valid string. The ``set_default_datarepo`` method needs to be used in order to access the datasets stored in the specific datarepo.

In [ ]:
### Can use "Health Byte - Dev"

datarepo_name = <DATAREPO_NAME> 
a360ai.set_default_datarepo(datarepo_name)

List the datasets stored in the current datarepo.

In [ ]:
datasets = a360ai.list_datasets()

if datasets.shape[0] == 0:
    raise Exception(f"No datasets associated with the datarepo {datarepo_name}. Please add a dataset.")
datasets

#### Select the model type

Please specify the type of task to be performed. Options are limited to "classification" and "regression".

In [ ]:
model_type = <MODEL_TYPE> 

#### Select the dataset
Please set the *dataset_name* variable to a valid string. 

In [ ]:
### The datasets "merged_anonymous.parquet" (classification) or 
### "machine_dataset.csv" (regression) can be used.
### Both datasets are in "Health Byte - Dev" datarepo.

dataset_name = <DATASET_NAME> 

df = a360ai.load_dataset(dataset_name)

#### Specify the target column 
Please set the target column out of the following columns

In [ ]:
df.columns.values.tolist()

In [ ]:
### "lifestyle" if using "merged_anonymous.parquet" 
### or "published_relative_performance" if using "machine_dataset.csv"

target_column = <TARGET_COLUMN> 

#### Please provide the number of classes
If the model_type is classification, please specify the number of classes present. An integer should be assigned to the variable ``num_classes``.

In [ ]:
# 3 if using "merged_anonymous.parquet"

num_classes = <NUM_CLASSES>

#### Split the dataset into train/test data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

X, y = df.drop(target_column,axis=1), df[target_column]

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, test_size=0.1)
   
num_features = X_train.shape[1]

### A360AI Model

Create/retrieve an A360AI model. The following arguments need to be passed to the ``get_or_create_model`` method: model_name, version and model_type (default is "classification").

In [ ]:
from uuid import uuid4
unique = str(uuid4())[-6:]

model = a360ai.get_or_create_model(
    model_name=f"Sample Notebook {flavor} {model_type} - {unique}", 
    version="1.0", 
    model_type=model_type
)

### A360AI Experiment

Create/retrieve an A360AI experiment. The following arguments need to be passed to the ``get_or_create_experiment`` method of the Model object: experiment_name, model_flavor, train_features and train_target. 

In [ ]:
experiment = model.get_or_create_experiment(
    experiment_name=f"sample_notebook_experiment{unique}",
    model_flavor = flavor, 
    train_features = X_train,
    train_target = y_train
)

#### View the experiments associated with the model

In [ ]:
model.list_experiments()

#### Choose the candidate model to be trained.

This model will be trained on the train dataset and evaluated on the test dataset. The candidate model will also be logged later after it has been trained.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
if model_type == "classification":
    candidate_model = RandomForestClassifier()
else:
    candidate_model = RandomForestRegressor()
    
scorer = candidate_model.score

### Create an A360AI Run

Use the Run to log the trained model, metadata, metrics and hyperparameters.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error

with experiment.run_experiment() as run:
    candidate_model.fit(X_train, y_train)

    run.log_metadata({
        "notes": "This model was generated with extreme care."
    })
    run.log_metrics({
        "train_score": scorer(X_train, y_train),
        "test_score": scorer(X_test, y_test),
    })
    run.log_hyperparameters({
        "foo": 1
    })
    run.log_model(candidate_model)

#### View the runs associated with the experiment. 
If the code above has been executed successfully, there will be a new entry in the dataframe returned by the ``list_runs`` method.

In [ ]:
experiment.list_runs()